# Projet : RAG LLM 

In [5]:
import os
import getpass
from langchain import hub
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings
from langchain_cohere import ChatCohere
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display

In [16]:
from dotenv import load_dotenv
import os

load_dotenv()
key = os.getenv("COHERE_API_KEY")
 
documents = []
for file_path in ["C:/Users/bouba/Nouvelle_vie/Projet_RAG_LLM/Data/old_cv.pdf"]: 
    loader =PyPDFLoader(file_path)
    documents.extend(loader.load())

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

embeddings = CohereEmbeddings(cohere_api_key=key, model="embed-english-v2.0")
vectorstore = Chroma.from_documents(splits, embeddings)

retriever = vectorstore.as_retriever()


In [21]:
cv_prompt_template = """
Vous allez générer un CV orienté Data Science à partir des informations suivantes. Organisez les informations de manière claire et professionnelle.

Voici les sections à inclure :
      DIALLO Alpha 
1. **Expérience professionnelle**
    - Inclure toutes les experiences 
    - Résumer chaque expérience en 2-3 points.
2. **Compétences techniques**
    - Liste des outils, langages et frameworks spécifiques à la Data Science.
    - Inclure des compétences générales comme "résolution de problèmes", "analyse statistique", etc.
3. **Formation**
    - Indiquer tous les diplômes et certificats obtenus. Je suis actuellement en Master 1.
    - Préciser les cours ou spécialisations liés à la Data Science.
4. **Projets**
    - Décrire brièvement 2-3 projets pertinents (par exemple, "Prédiction des ventes avec Machine Learning").
5. **Autres informations**
    - Langues parlées, hobbies, ou toute autre information pertinente.

6. **Soft skills**
    - Rajoute des soft skills.

Sections de texte récupérées : {context}

Générez un CV structuré et concis avec des sous-sections claires et des listes à puces. Utilisez un style professionnel et soigné.

"""
prompt = PromptTemplate(input_variables=["context"], template=cv_prompt_template)


In [23]:
llm = ChatCohere(model="command-r-plus", temperature=0.3) 
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    retriever | format_docs 
    | prompt  
    | llm 
)

cv = rag_chain.invoke("Générer un nouveau CV orienté Data Science")
cv_text = cv.content  

with open("output.md", "w", encoding="utf-8") as file:
    file.write(f"# CV généré\n\n{cv_text}")

display(Markdown(f"### CV généré\n{cv_text}"))
# print("CV généré\n",cv_text)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


### CV généré
Bien sûr ! Voici un CV structuré et professionnel pour Diallo Alpha :

# CV de Diallo Alpha

## Data Scientist en Alternance

**Adresse :** 56 Boulevard de Brandebourg, Ivry-sur-Seine (94200), France

**Téléphone :** 06 67 68 18 05

**Email :** boubacarporeck2000@gmail.com

## Profil

Je suis actuellement étudiant en Master 1 avec l'ambition de poursuivre en Master 2 l'année prochaine. Je suis à la recherche d'une opportunité d'alternance dans le domaine de la Data Science à partir de septembre 2025. Mon objectif est de mettre en pratique mes compétences techniques et analytiques pour contribuer à la valeur ajoutée de votre entreprise.

## Expérience professionnelle

**Moniteur de Bibliothèques (job étudiant)**
Octobre 2023 - Juin 2024
Université Paris Est Créteil, Paris
- Accueil et assistance des utilisateurs, fourniture de renseignements bibliographiques et orientation dans la salle de lecture.
- Gestion du prêt informatisé des documents et entretien des espaces de travail.
- Maintenance de base des équipements de reprographie.

**Assistant d'Education (job étudiant)**
Depuis Octobre 2023
Lycée Guillaume Apollinaire, Thiais, Paris
- Encadrement des élèves pendant les études, les récréations, les repas et dans les locaux scolaires.
- Surveillance des contrôles et des examens, veillant au respect des règles et contrôlant l'assiduité.

## Compétences techniques

**Outils et langages :**
- Python, R, SAS : maîtrise des logiciels d'analyse de données.
- Mask-RCNN : entraînement de modèles de reconnaissance d'objets.

**Compétences en Data Science :**
- Extraction et analyse de données.
- Algorithmes et Machine Learning.
- Résolution de problèmes complexes.
- Analyse statistique et interprétation des résultats.

## Formation

**Master 1 MASERATI & GPIA**
Septembre 2024 - présent
Université Paris Est Créteil, Créteil

**Licence 3 Economie Gestion**
Septembre 2023 - Juin 2024
Université Paris Est Créteil, Créteil

**DEUG Economie Gestion**
Septembre 2022 - Juin 2023
Ecole d'économie, Clermont-Ferrand, France

**Licence Economie-Finance**
Octobre 2019 - Juillet 2022
Université Lansana Conté, Conakry, Guinée

## Projets

**Prédiction des ventes avec Machine Learning :** Développement d'un modèle de prédiction des ventes en utilisant des techniques de Machine Learning pour une entreprise de commerce en ligne.

**Analyse de données pour l'ONG environnementale :** Collaboration avec une ONG environnementale pour analyser les données de pollution et proposer des stratégies d'atténuation.

## Autres informations

**Langues :** Anglais (B2), Français (langue maternelle).

**Centres d'intérêt :** Présidence de l'ONG environnementale Save ONG, lecture, voyages.

## Soft Skills

- Rigoureux et organisé.
- Dynamique et autonome.
- Excellentes capacités de communication orale et écrite.

Ce CV met en valeur les expériences, compétences techniques et formations de Diallo Alpha, tout en soulignant ses projets pertinents et ses soft skills. N'hésitez pas à me dire si vous souhaitez apporter des modifications ou des ajouts à ce CV.

## Remarque : 
Le CV est précis et réaliste, respectant fidèlement votre situation actuelle, notamment votre statut en Master 1. La structure est claire, et les expériences ainsi que les compétences sont bien mises en avant. Cependant, le ton reste très factuel et manque légèrement de créativité ou de détails engageants

### Maintenant prenons temperature = 0.7

In [24]:
llm = ChatCohere(model="command-r-plus", temperature=0.7)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    retriever | format_docs 
    | prompt  
    | llm 
)

cv = rag_chain.invoke("Générer un nouveau CV orienté Data Science")
cv_text = cv.content  

display(Markdown(f"### CV généré\n{cv_text}"))
# print("CV généré\n",cv_text)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


### CV généré
Bien sûr ! Voici un CV structuré et professionnel pour Diallo Alpha, mettant en valeur ses expériences et compétences en Data Science :

# CV - Diallo Alpha

## Data Scientist en Alternance

### Coordonnées

- Adresse : 56 Boulevard de Brandebourg, Ivry-sur-Seine (94200), France
- Téléphone : 06 67 68 18 05
- Email : boubacarporeck2000@gmail.com

### Profil

Étudiante en Master 2, je suis à la recherche d'une alternance à partir de septembre 2025 dans le domaine de la Data Science. Passionnée par l'analyse de données et l'intelligence artificielle, mon objectif est de mettre en pratique mes compétences et de contribuer à la création de valeur au sein d'une entreprise innovante.

### Expérience professionnelle

**Moniteur de Bibliothèques (job étudiant)**
Octobre 2023 - Juin 2024
Université Paris Est Créteil, Paris
- Accueil et information du public, orientation en salle de lecture
- Gestion des prêts et du rangement des documents
- Entretien des équipements de reprographie
- Garantie d'un environnement de travail calme

**Assistant d'Education (job étudiant)**
Depuis Octobre 2023
Lycée Guillaume Apollinaire, Thiais, Paris
- Encadrement des élèves lors des études, réfectoire, récréations, etc.
- Surveillance des contrôles et examens
- Application du règlement intérieur et contrôle de l'assiduité

### Compétences techniques

**Outils et langages :**
- Python, R, SAS
- Mask-RCNN pour la reconnaissance d'objets
- Logiciels d'analyse de données

**Compétences en Data Science :**
- Extraction et analyse de données
- Algorithmes et Machine Learning
- Résolution de problèmes complexes
- Analyse statistique et interprétation des résultats

### Formation

**Master 1 MASERATI & GPIA**
Septembre 2024 - Présent
Université Paris Est Créteil, Créteil

**Licence 3 Economie Gestion**
Septembre 2023 - Juin 2024
Université Paris Est Créteil, Créteil

**DEUG Economie Gestion**
Septembre 2022 - Juin 2023
Ecole d'économie, Clermont-Ferrand, France

**Licence Economie-Finance**
Octobre 2019 - Juillet 2022
Université Lansana Conté, Conakry, Guinée

### Projets pertinents

**Prédiction des ventes avec Machine Learning**
Développement d'un modèle de prédiction des ventes en utilisant des techniques de Machine Learning pour une entreprise de commerce en ligne.

**Analyse de données pour l'optimisation des prix**
Analyse des données de marché pour aider une startup à optimiser ses stratégies de prix et améliorer ses parts de marché.

### Soft Skills

- Rigoureux, organisé et autonome
- Excellentes capacités de communication orale et écrite
- Capacité à travailler en équipe et à gérer des projets

### Langues et Centres d'intérêt

**Langues :**
- Anglais : Niveau B2

**Centres d'intérêt :**
- Président de l'ONG Save ONG pour la protection de l'environnement
- Lecture, voyages et découverte de nouvelles cultures

Ce CV met en valeur les expériences et compétences de Diallo Alpha en Data Science, tout en présentant ses soft skills et ses centres d'intérêt. N'hésitez pas à me dire si vous souhaitez apporter des modifications ou ajouts supplémentaires !

## Remarque : 
Ce CV est plus engageant et inclut des éléments créatifs comme des projets supplémentaires. Cependant, il introduit des erreurs factuelles, notamment en vous positionnant en Master 2 au lieu de Master 1. Le ton est plus dynamique et professionnel, mais cette température commence à s’éloigner légèrement de la réalité.

### Essayons temperature = 0.9

In [27]:
llm = ChatCohere(model="command-r-plus", temperature=0.9) 
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    retriever | format_docs 
    | prompt  
    | llm 
)

cv = rag_chain.invoke("Générer un nouveau CV orienté Data Science")
cv_text = cv.content  

display(Markdown(f"### CV généré\n{cv_text}"))
# print("CV généré\n",cv_text)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


### CV généré
Bien sûr ! Voici un CV structuré et professionnel pour Diallo Alpha :

# Diallo Alpha

## Coordonnées

- Adresse : 56 Boulevard de Brandebourg, Ivry-sur-Seine (94200), France
- Téléphone : 06 67 68 18 05
- Email : boubacarporeck2000@gmail.com

## Objectif

Data Scientist en Alternance

Je suis actuellement en Master 1 et je recherche, dans le cadre de mon Master 2 l'année prochaine, une entreprise pour une alternance à partir de septembre 2025. Mon objectif est de mettre en pratique mes compétences en Data Science et d'apporter une valeur ajoutée à votre organisation.

## Expérience professionnelle

**Moniteur de Bibliothèques (job étudiant)**
Octobre 2023 - Juin 2024
Université Paris Est Créteil, Paris

- Accueil et assistance des utilisateurs, fourniture de renseignements bibliographiques et orientation dans la salle de lecture.
- Rangement des documents, gestion des espaces de groupe et prêt informatisé.
- Entretien des équipements de reprographie et maintien d'un environnement de travail calme.
- Responsable de la fermeture de la bibliothèque, y compris le rangement et la sécurité.

**Assistant d'Éducation (job étudiant)**
Depuis Octobre 2023
Lycée Guillaume Apollinaire, Thiais, Paris

- Encadrement des élèves pendant les études, les récréations, les repas et dans les locaux scolaires.
- Surveillance pendant les contrôles et les examens, en veillant au respect des règles.
- Contrôle de l'assiduité et maintien de la discipline.

## Compétences techniques

- Reconnaissance d'objets avec Mask-RCNN.
- Logiciels d'analyse : Python, R et SAS.
- Extraction et analyse de données.
- Algorithmes et Machine Learning.
- Résolution de problèmes et analyse statistique.

## Formation

**Master 1 MASERATI & GPIA**
Depuis Septembre 2024
Université Paris Est Créteil, Créteil

**Licence 3 Économie Gestion**
Septembre 2023 - Juin 2024
Université Paris Est Créteil, Créteil

**DEUG Économie Gestion**
Septembre 2022 - Juin 2023
Ecole d'économie, Clermont-Ferrand, France

**Licence Économie-Finance**
Octobre 2019 - Juillet 2022
Université Lansana Conté, Conakry, Guinée

## Projets

- Prédiction des ventes avec Machine Learning : développement d'un modèle prédictif pour les ventes d'une entreprise, en utilisant des techniques de Machine Learning.
- Analyse de données pour l'optimisation des prix : extraction et analyse de données pour aider une entreprise à optimiser sa stratégie de prix.
- Reconnaissance d'objets avec Mask-RCNN : entraînement d'un modèle de Deep Learning pour la reconnaissance d'objets dans des images.

## Autres informations

**Langues** : Anglais (niveau B2)

**Centres d'intérêt** : Environnement (président de l'ONG Save), lecture et voyages.

## Soft Skills

- Rigoureux, organisé et autonome.
- Excellentes capacités de communication orale et écrite.
- Dynamique et proactif.
- Capacité à travailler en équipe et à gérer plusieurs tâches simultanément.

Ce CV met en valeur les compétences et l'expérience de Diallo Alpha, en les organisant de manière claire et concise. N'hésitez pas à me dire si vous souhaitez apporter des modifications ou des ajouts !

## Remarque 
Le CV devient très dynamique et riche en contenu, avec des détails amplifiés sur les projets et compétences. Il met fortement en avant des qualités personnelles, mais s’éloigne davantage de la précision, notamment avec des informations incorrectes sur le niveau académique. Ce ton peut être attractif mais peut poser des problèmes de crédibilité.